[爬蟲案例分享]
- 難點：

    1.資料量大，但每個查詢最多150個分頁，因此需要將查詢的範圍限縮。
    解決方式是在查詢時組合地區與職務類型，進行細緻的查詢。

    2.雖然幾乎沒有設定反爬蟲的機制，但每個 request 送的速度太快仍然會有失敗的狀況發生
    用while迴圈，當request成功是繼續前往下一頁爬蟲，request成功但資料小於20筆，查詢下一個組合，request等於20，表示還有資料需要繼續抓下一個分頁的資料。

    3.因為資料量大，建議按照組別存一次資料，這樣遇到錯誤時才不用重新爬資料。

    
- 因為查詢職缺最多有20\*150=3000筆資料，所以要將查詢的的條件透過條件進行篩選、排列組合，藉此撈出完整的資料
- 由公司資訊：地址，聯絡電話、職務
- 縣市和職務的分類有個資的代號，但中間有經過變(如縣市合併)更並不是單純的序列，因此要花時間找一下
- 可以看各個縣市的職缺數量與職務類型，進一步分析需要什麼技能
- 你要在哪裡，需要什麼技能、待遇如何
- 可以去串各縣市的人口數、教育程度等資訊
- 縣市的產業結構
- 可行的分析項目

In [52]:
# Tips in this project
# multithreads could miss a lot og data, so we need to check the numbers of data we collect

# 載入套件

In [2]:
# !jupyter trust HumanResource_104_v2.ipynb

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import re
import time
import os
import gc
from IPython.display import clear_output
from tomorrow import threads
import pickle
# import grequests

In [2]:
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36',
         'Referer': 'https://www.104.com.tw/',
         'Host': 'www.104.com.tw',
         'DNT':'1',
         'Cookie':'GoogleBot'}

# 職缺清單
- 透過地區與職務分類來請求職缺資料

In [3]:
# Area code
url = 'https://static.104.com.tw/category-tool/json/Area.json'
resp = requests.get(url, headers=headers)
df1 = pd.read_json(resp.text)

df1 = df1.explode('n')
df1['des2'] = df1['n'].apply(lambda x:x['des'])
df1['no2'] = df1['n'].apply(lambda x:x['no'])
df1['n'] = df1['n'].apply(lambda x:x['n'])

df1 = df1.explode('n')
df1['des3'] = df1['n'].apply(lambda x:x['des'])
df1['no3'] = df1['n'].apply(lambda x:x['no'])
df1 = df1.drop('n', axis=1)

# filter data
df1 = df1.loc[df1['no']==6001000000]

df1.head(3)

,des,no,des2,no2,des3,no3
0,台灣地區,6001000000,台北市,6001001000,台北市中正區,6001001001
0,台灣地區,6001000000,台北市,6001001000,台北市大同區,6001001002
0,台灣地區,6001000000,台北市,6001001000,台北市中山區,6001001003


In [4]:
url= 'https://static.104.com.tw/category-tool/json/JobCat.json'
resp = requests.get(url, headers=headers)
df2 = pd.read_json(resp.text)

df2 = df2.explode('n')
df2['des2'] = df2['n'].apply(lambda x:x['des'])
df2['no2'] = df2['n'].apply(lambda x:x['no'])
df2['n'] = df2['n'].apply(lambda x:x['n'])

# drop too detail job type
# df2 = df2.explode('n')
# df2['des3'] = df2['n'].apply(lambda x:x['des'])
# df2['no3'] = df2['n'].apply(lambda x:x['no'])
df2 = df2.drop('n', axis=1)
df2.head(3)

,des,no,des2,no2
0,經營╱人資類,2001000000,經營╱幕僚類人員,2001001000
0,經營╱人資類,2001000000,人力資源類人員,2001002000
1,行政╱總務╱法務類,2002000000,行政╱總務類人員,2002001000


In [5]:
# Last executed at 2021-06-19 11:07:26 in 1h 49m 36s
jobs = []
for area, areacode in zip(df1['des3'],df1['no3']):
    for jobtype, jobtypecode in zip(df2['des2'], df2['no2']):
        ndf = []
        page = 1 ########################Tip: we don't need to crawl 100 pages by each type & area  
        while page < 100: ######################################### Tips: while loop
            try:
                ############################################## Tips: ro=1為全職, isnew=31表示近31天有更新(表示職缺有效)
                url = 'https://www.104.com.tw/jobs/search/list?ro=1&isnew=30&jobcat={}&jobcatExpansionType=1&area={}&order=11&asc=0&page={}&mode=l&jobsource=2018indexpoc'.format(jobtypecode, areacode, page) 
                resp = requests.get(url, headers=headers, timeout=5)
                tmp_df = pd.DataFrame(resp.json()['data']['list'])
                ndf.append(tmp_df)
                page += 1
                if tmp_df.shape[0]<30:
                    break
            except:
                print('Retry')
        
        ndf = pd.concat(ndf)
        print('There are {} {} jobs in {}'.format(ndf.shape[0], jobtype, area))
        jobs.append(ndf)
#     break
jobs = pd.concat(jobs, ignore_index=True)

There are 281 經營╱幕僚類人員 jobs in 台北市中正區
There are 106 人力資源類人員 jobs in 台北市中正區
There are 552 行政╱總務類人員 jobs in 台北市中正區
There are 80 法務╱智財類人員 jobs in 台北市中正區
There are 357 金融專業相關類人員 jobs in 台北市中正區
There are 317 財務╱會計╱稅務類 jobs in 台北市中正區
There are 803 行銷類人員 jobs in 台北市中正區
There are 187 產品企劃類人員 jobs in 台北市中正區
There are 278 專案╱產品管理類人員 jobs in 台北市中正區
There are 274 客戶服務類人員 jobs in 台北市中正區
There are 650 門市營業類人員 jobs in 台北市中正區
There are 1013 業務銷售類人員 jobs in 台北市中正區
There are 66 貿易類人員 jobs in 台北市中正區
There are 419 餐飲類人員 jobs in 台北市中正區
There are 126 旅遊休閒類人員 jobs in 台北市中正區
There are 108 美容╱美髮類人員 jobs in 台北市中正區
There are 686 軟體╱工程類人員 jobs in 台北市中正區
There are 297 MIS╱網管類人員 jobs in 台北市中正區
There are 33 操作╱技術類人員 jobs in 台北市中正區
There are 129 維修╱技術服務類人員 jobs in 台北市中正區
There are 76 採購╱資材╱倉管類人員 jobs in 台北市中正區
There are 45 運輸物流類人員 jobs in 台北市中正區
There are 138 營建規劃類人員 jobs in 台北市中正區
There are 88 營建施作類人員 jobs in 台北市中正區
There are 134 製圖╱測量類人員 jobs in 台北市中正區
There are 234 設計類人員 jobs in 台北市中正區
There are 89 傳播藝術類人員 jobs in

In [6]:
jobs.to_pickle('./data/job_abs.pkl')
jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443850 entries, 0 to 443849
Data columns (total 39 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   jobType         443850 non-null  object
 1   jobNo           443850 non-null  object
 2   jobName         443850 non-null  object
 3   jobNameSnippet  443850 non-null  object
 4   jobRole         443850 non-null  object
 5   jobRo           443850 non-null  object
 6   jobAddrNo       443850 non-null  object
 7   jobAddrNoDesc   443850 non-null  object
 8   jobAddress      443850 non-null  object
 9   description     443850 non-null  object
 10  optionEdu       443850 non-null  object
 11  period          443850 non-null  object
 12  periodDesc      443850 non-null  object
 13  applyCnt        443850 non-null  object
 14  applyDesc       443850 non-null  object
 15  custNo          443850 non-null  object
 16  custName        443850 non-null  object
 17  coIndustry      443850 non-nu

# 職務內容
會有重複的職缺，distinct by link

## requests

In [7]:
jobs['link'][0]

{'applyAnalyze': '//www.104.com.tw/jobs/apply/analysis/31fsa?channel=104rpt&jobsource=hotjob_chr',
 'job': '//www.104.com.tw/job/31fsa?jobsource=hotjob_chr',
 'cust': '//www.104.com.tw/company/13trrwr4?jobsource=hotjob_chr'}

In [8]:
jobs = pd.read_pickle('./data/job_abs.pkl')
jobs['code'] = jobs['link'].apply(lambda x: x['job'].split('/')[-1].split(r'?')[0])
jobs['ajax'] = jobs['code'].apply(lambda x: 'https://www.104.com.tw/job/ajax/content/' + x)

print(jobs.shape)
jobs = jobs.drop_duplicates(['code'])
print(jobs.shape)
jobs.head(3)

(443850, 41)
(271663, 41)


,jobType,jobNo,jobName,jobNameSnippet,jobRole,jobRo,jobAddrNo,jobAddrNoDesc,jobAddress,description,...,landmark,link,jobsource,jobNameRaw,custNameRaw,lon,lat,remoteWorkType,code,ajax
0,1,5105962,業務主任,業務主任,1,1,6001001001,台北市中正區,南昌路二段70號4樓之7,1.故宮/中研院期刊庫/全球頂尖智庫(社科類資料庫)行銷業務\n2.各級學校.圖書館.研究機...,...,距捷運古亭站150公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,hotjob_chr,業務主任,聯合百科電子出版有限公司,121.521357,25.026384,0,31fsa,https://www.104.com.tw/job/ajax/content/31fsa
1,1,10122438,【保障薪獎5萬起+高抽成獎金】_業務儲備主管M,【保障薪獎5萬起+高抽成獎金】_業務儲備主管M,1,1,6001001001,台北市中正區,和平西路一段5號1樓（捷運古亭站8號出口）,&quot;TutorABC為線上教育開創者，致力教育科技與英語學習發展逾23年，2020年...,...,距捷運古亭站130公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,hotjob_chr,【保障薪獎5萬起+高抽成獎金】_業務儲備主管M,TutorABC_麥奇數位股份有限公司,121.5217528,25.027139,0,60yiu,https://www.104.com.tw/job/ajax/content/60yiu
2,0,12346289,管理部副理,管理部副理,1,1,6001001001,台北市中正區,中山北路一段2號4樓之2,1.作業協調：與香港、新加坡窗口就日常事項溝通協調 \n2.金融商品(股票、基金、結構型商...,...,距捷運台北車站320公尺,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,2018indexpoc,管理部副理,宇宸財富管理顧問有限公司,121.5194582,25.0460358,2,7cmgh,https://www.104.com.tw/job/ajax/content/7cmgh


In [29]:
@threads(40)
def MultiThread_Crawler(ajax, code):
    try:
        # Tips: Don't keep you date in the memory!
        data = requests.get(ajax, headers=headers).json()['data']
        with open('data/JobDetail/' + code + '.pickle', 'wb') as job:
            pickle.dump(data, job)
    except:
        pass

In [42]:
crawled_list = pd.DataFrame(data=os.listdir(r'C:\Users\TLYu0419\Documents\Github\DataScience\02_Data_Collenction\WebCrawler\HumanResource_104\data\JobDetail'), columns=['code'])
crawled_list['code'] = crawled_list['code'].apply(lambda x: re.sub('.pickle', '', x))
jobs2 = pd.merge(jobs, crawled_list, how='outer', on='code', indicator=True)
jobs2 = jobs2.loc[jobs2['_merge']=='left_only']
jobs2

,jobType,jobNo,jobName,jobNameSnippet,jobRole,jobRo,jobAddrNo,jobAddrNoDesc,jobAddress,description,...,link,jobsource,jobNameRaw,custNameRaw,lon,lat,remoteWorkType,code,ajax,_merge
256557,0,12355416,日洋風居酒屋【千夏】儲備幹部/外場正職人員,日洋風居酒屋【千夏】儲備幹部/外場正職人員,1,1,6001016011,高雄市左營區,文康路40號1樓,"\n-公司介紹-\n\n左營漢神巨蛋旁「千夏居酒屋」\n由日本師傅經營,提供創意日洋風餐酒館...",...,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,2018indexpoc,日洋風居酒屋【千夏】儲備幹部/外場正職人員,千夏居酒屋,120.3036565,22.6730073,0,7cti0,https://www.104.com.tw/job/ajax/content/7cti0,left_only
256576,0,12353158,日洋風居酒屋【千夏】儲備幹部/外場正職人員,日洋風居酒屋【千夏】儲備幹部/外場正職人員,1,1,6001016011,高雄市左營區,文康路40號1樓,"\n-公司介紹-\n\n左營漢神巨蛋旁「千夏居酒屋」\n由日本師傅經營,提供創意日洋風餐酒館...",...,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,2018indexpoc,日洋風居酒屋【千夏】儲備幹部/外場正職人員,千夏居酒屋,120.3036565,22.6730073,0,7crra,https://www.104.com.tw/job/ajax/content/7crra,left_only
256582,0,12351903,日洋風居酒屋【千夏】儲備幹部/外場正職人員,日洋風居酒屋【千夏】儲備幹部/外場正職人員,1,1,6001016011,高雄市左營區,文康路40號1樓,"\n-公司介紹-\n\n左營漢神巨蛋旁「千夏居酒屋」\n由日本師傅經營,提供創意日洋風餐酒館...",...,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,2018indexpoc,日洋風居酒屋【千夏】儲備幹部/外場正職人員,千夏居酒屋,120.3036565,22.6730073,0,7cqsf,https://www.104.com.tw/job/ajax/content/7cqsf,left_only
257761,0,12355412,日洋風居酒屋【千夏】儲備幹部/外場正職人員,日洋風居酒屋【千夏】儲備幹部/外場正職人員,1,1,6001016011,高雄市左營區,文康路40號1樓,"\n-公司介紹-\n\n左營漢神巨蛋旁「千夏居酒屋」\n由日本師傅經營,提供創意日洋風餐酒館...",...,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,2018indexpoc,日洋風居酒屋【千夏】儲備幹部/外場正職人員,千夏居酒屋,120.3036565,22.6730073,0,7cthw,https://www.104.com.tw/job/ajax/content/7cthw,left_only
257798,0,12353156,日洋風居酒屋【千夏】儲備幹部/外場正職人員,日洋風居酒屋【千夏】儲備幹部/外場正職人員,1,1,6001016011,高雄市左營區,文康路40號1樓,"\n-公司介紹-\n\n左營漢神巨蛋旁「千夏居酒屋」\n由日本師傅經營,提供創意日洋風餐酒館...",...,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,2018indexpoc,日洋風居酒屋【千夏】儲備幹部/外場正職人員,千夏居酒屋,120.3036565,22.6730073,0,7crr8,https://www.104.com.tw/job/ajax/content/7crr8,left_only
257809,0,12351900,日洋風居酒屋【千夏】儲備幹部/外場正職人員,日洋風居酒屋【千夏】儲備幹部/外場正職人員,1,1,6001016011,高雄市左營區,文康路40號1樓,"\n-公司介紹-\n\n左營漢神巨蛋旁「千夏居酒屋」\n由日本師傅經營,提供創意日洋風餐酒館...",...,{'applyAnalyze': '//www.104.com.tw/jobs/apply/...,2018indexpoc,日洋風居酒屋【千夏】儲備幹部/外場正職人員,千夏居酒屋,120.3036565,22.6730073,0,7cqsc,https://www.104.com.tw/job/ajax/content/7cqsc,left_only


In [43]:
# [MultiThread_Crawler(link, code) for link, code in zip(jobs['ajax'], jobs['code'])]
[MultiThread_Crawler(link, code) for link, code in zip(jobs2['ajax'], jobs2['code'])]

## parse

- 評估職務類別要不要保留編號

In [3]:
len(os.listdir('data/JobDetail'))

271657

In [4]:
data = pd.read_pickle('data/JobDetail/'+ os.listdir('data/JobDetail')[12])
data.keys()

dict_keys(['corpImageRight', 'header', 'contact', 'environmentPic', 'condition', 'welfare', 'jobDetail', 'switch', 'custLogo', 'postalCode', 'closeDate', 'industry', 'custNo', 'reportUrl', 'industryNo', 'employees', 'chinaCorp'])

In [5]:
df = []
for i, filename in enumerate(os.listdir('data/JobDetail')):
    if i % 5000 == 0:
        print(i)
    try:
        data = pd.read_pickle('data/JobDetail/'+ filename)
        
        ndf = pd.DataFrame([{
            ############ 摘要 ############
            'jobName': data['header']['jobName'],  # 職稱
            'appearDate':data['header']['appearDate'],  # 刊登日期
            'custName':data['header']['custName'],  # 公司名稱
            'custUrl':data['header']['custUrl'],  # 公司網址
            'analysisUrl':data['header']['analysisUrl'], # 職缺分析
                             
            ############ 聯絡方式 ############
            'hrName':data['contact']['hrName'],  # 聯絡人姓名
            'email':data['contact']['email'],  # Mail
            'visit':data['contact']['visit'],  # 地址
            'phone':data['contact']['phone'],  # 電話
                             
            ############ 條件要求 ############
            'acceptRole':data['condition']['acceptRole']['role'], # 接受身份
            'workExp':data['condition']['workExp'],  # 工作經歷
            'edu':data['condition']['edu'],  # 教育程度
            'major':data['condition']['major'], # 科系要求
            'language':data['condition']['language'],  # 語言
            'specialty':data['condition']['specialty'], # 擅長工具
            'skill':data['condition']['skill'], # 工作技能
            'certificate':data['condition']['certificate'], # 具備證照
            'other':data['condition']['other'],  # 其他條件
                             
#             ############ 福利制度 ############
#             'welfare_tag':data['welfare']['tag'], # 其他福利
#             'welfare':data['welfare']['welfare'],  # 福利制度
#             'legalTag':data['welfare']['legalTag'],  # 法定項目
            
            ############ 工作內容 ############
            'jobDescription':data['jobDetail']['jobDescription'], # 工作內容
            'jobCategory': data['jobDetail']['jobCategory'], # 職務類別
            'salary':data['jobDetail']['salary'],  # 工作待遇
            'salaryMin':data['jobDetail']['salaryMin'],  # 最低待遇
            'salaryMax':data['jobDetail']['salaryMax'],  # 最高待遇
            'salaryType':data['jobDetail']['salaryType'],# 月薪/日薪/時薪
            'jobType':data['jobDetail']['jobType'],  # 全職/兼職
            'workType':data['jobDetail']['workType'],  # 短期/長期工讀標籤
            'addressNo':data['jobDetail']['addressNo'], # 縣市/區域
            'addressRegion':data['jobDetail']['addressRegion'], # 縣市/區域
            'addressDetail':data['jobDetail']['addressDetail'], # 地址
            'industryArea':data['jobDetail']['industryArea'],
            'longitude':data['jobDetail']['longitude'],  # 經度
            'latitude':data['jobDetail']['latitude'],  # 維度
#             'manageResp':data['jobDetail']['manageResp'],  # 管理職
#             'businessTrip':data['jobDetail']['businessTrip'],# 是否需出差
#             'workPeriod':data['jobDetail']['workPeriod'], # 班別，日/夜/輪班
#             'vacationPolicy':data['jobDetail']['vacationPolicy'], # 周休/排休...
#             'startWorkingDay':data['jobDetail']['startWorkingDay'], # 可上班日
#             'hireType':data['jobDetail']['hireType'],  # 不確定是不是派遣??
#             'delegatedRecruit':data['jobDetail']['delegatedRecruit'], # ??
#             'needEmp':data['jobDetail']['needEmp'],  # 需求人數
#             'switch':data['switch'],  # ??
            ############ 產業 ############
            'industry':data['industry'],  # 產業
            'industryNo':data['industryNo'], # 產業編號
            'employees':data['employees'] # 員工人數
                             }])
        df.append(ndf)
    except:
        print(data)
df = pd.concat(df, ignore_index=True)
df

0
{'header': {'jobName': '出貨助理/會計助理', 'custName': '銳德利企業有限公司'}, 'switch': 'off', 'custNo': '79980979000'}
5000
{'header': {'jobName': '記/查帳人員(有企圖心成為會計師者)', 'custName': '遠見國際會計師事務所'}, 'switch': 'off', 'custNo': '26259968000'}
{'header': {'jobName': '全職牙醫助理', 'custName': '鼎極牙醫診所'}, 'switch': 'off', 'custNo': '26675417000'}
10000
{'header': {'jobName': '業務人員(台中)', 'custName': '和成欣業股份有限公司'}, 'switch': 'off', 'custNo': '3228304000'}
{'header': {'jobName': '會計人員', 'custName': '鏮達實業股份有限公司'}, 'switch': 'off', 'custNo': '86563921000'}
{'header': {'jobName': '護士/護理師', 'custName': '蔡定洲耳鼻喉科診所'}, 'switch': 'off', 'custNo': '26072730000'}
15000
{'header': {'jobName': 'i智慧經紀人 (保障新人前九個月每月5萬元!) I84', 'custName': '永慶房產集團_永慶房屋仲介股份有限公司'}, 'switch': 'off', 'custNo': '22760834000'}
{'header': {'jobName': 'i智慧經紀人 (保障新人前九個月每月5萬元!) I82', 'custName': '永慶房產集團_永慶房屋仲介股份有限公司'}, 'switch': 'off', 'custNo': '22760834000'}
{'header': {'jobName': 'i智慧經紀人 (保障新人前九個月每月5萬元!) I94', 'custName': '永慶房產集團_永慶房屋仲介股份有限公司'}, 'switch

,jobName,appearDate,custName,custUrl,analysisUrl,hrName,email,visit,phone,acceptRole,...,workType,addressNo,addressRegion,addressDetail,industryArea,longitude,latitude,industry,industryNo,employees
0,"生產線作業員(起薪$30,000，含餐食) (加班、獎金另計)",2021/08/16,總信食品有限公司,https://www.104.com.tw/company/13vdp2c8,//www.104.com.tw/jobs/apply/analysis/10062,胡慧敏小姐,,,,"[{'code': 1, 'description': '上班族'}, {'code': 2...",...,[],6001008008,台中市南屯區,工業區二十二路17號,台中工業區,120.606144,24.160068,食品什貨批發業,1003001002,95人
1,汐止分院 護理科 護理人員(內外科病房.產兒科病房.加護病房.門診.安寧病房.護理之家.急診),2021/08/17,國泰醫療財團法人國泰綜合醫院,https://www.104.com.tw/company/1waq7ko,//www.104.com.tw/jobs/apply/analysis/1006w,李小姐,sue@cgh.org.tw,,,"[{'code': 1, 'description': '上班族'}, {'code': 6...",...,[],6001002004,新北市汐止區,建成路59巷2號,,121.66141,25.072999,醫院,1012001001,3400人
2,印刷見習技師,2021/08/16,康揚特殊美術印刷股份有限公司,https://www.104.com.tw/company/5ql7jgg,//www.104.com.tw/jobs/apply/analysis/1008q,邱先生,ford521213@yahoo.com.tw,,,"[{'code': 1, 'description': '上班族'}, {'code': 2...",...,[],6001001011,台北市南港區,南港路三段50巷10號3樓,,121.5910051,25.0553784,印刷業,1002006002,35人
3,證券營業員(南科分公司),2021/08/12,國票綜合證券股份有限公司,https://www.104.com.tw/company/at8koao,//www.104.com.tw/jobs/apply/analysis/100il,郭經理,,,,"[{'code': 1, 'description': '上班族'}, {'code': 2...",...,[],6001014036,台南市新市區,',,120.3,23.0833333,證券及期貨業,1004002002,1056人
4,汐止分院 護理科 內外科 專科護理師,2021/08/17,國泰醫療財團法人國泰綜合醫院,https://www.104.com.tw/company/1waq7ko,//www.104.com.tw/jobs/apply/analysis/100ry,李小姐,sue@cgh.org.tw,,,"[{'code': 1, 'description': '上班族'}, {'code': 6...",...,[],6001002004,新北市汐止區,建成路59巷2號,,121.66141,25.072999,醫院,1012001001,3400人
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271457,初級查帳審計人員,2021/08/17,聯緯聯合會計師事務所,https://www.104.com.tw/company/19uizocg,//www.104.com.tw/jobs/apply/analysis/zygi,陳小姐,,,03-6673668,"[{'code': 1, 'description': '上班族'}, {'code': 2...",...,[],6001006002,新竹縣竹北市,勝利三街77號1樓,,121.0278526,24.8230679,會計服務業,1008002001,38人
271458,業務儲備幹部 (保障前9個月每月5萬元) I01,2021/08/18,永慶房產集團_永慶房屋仲介股份有限公司,https://www.104.com.tw/company/agf7nww,//www.104.com.tw/jobs/apply/analysis/zz37,袁小姐,,,,"[{'code': 1, 'description': '上班族'}, {'code': 2...",...,[],6001001006,台北市萬華區,依居住區域就近安排,,121.4970294,25.0262857,不動產經營業,1011004001,3000人
271459,塑膠設備操作員_磨粉機,2021/08/16,良輝塑膠工業股份有限公司,https://www.104.com.tw/company/vd2i754,//www.104.com.tw/jobs/apply/analysis/zzjx,鄭先生/潘小姐,,,06-2633110-31,"[{'code': 1, 'description': '上班族'}, {'code': 2...",...,[],6001014003,台南市南區,安平工業區新信路3號,安平工業區,120.182326,22.972978,塑膠製品製造業,1002009002,50人
271460,Java程式開發工程師,2021/08/17,藍科數位科技股份有限公司,https://www.104.com.tw/company/wjm2d5c,//www.104.com.tw/jobs/apply/analysis/zzsf,陳小姐,,,,"[{'code': 1, 'description': '上班族'}, {'code': 6...",...,[],6001001010,台北市內湖區,行愛路151號2樓之2,,121.5801053,25.0643078,電腦軟體服務業,1001001002,90人


# join 產業

In [6]:
# indust
url= 'https://static.104.com.tw/category-tool/json/Indust.json'
resp = requests.get(url, headers=headers)
df3 = pd.read_json(resp.text)

df3 = df3.explode('n')
df3['des2'] = df3['n'].apply(lambda x:x['des'])
df3['no2'] = df3['n'].apply(lambda x:x['no'])
df3['n'] = df3['n'].apply(lambda x:x['n'])

df3 = df3.explode('n')
df3['des3'] = df3['n'].apply(lambda x:x['des'])
df3['no3'] = df3['n'].apply(lambda x:x['no'])
df3 = df3.drop('n', axis=1)
df3 = df3.rename(columns={'des':'industry',
                          'no':'industryNo',
                          'des2':'industry2',
                          'no2':'industryNo2',
                          'des3':'industry3',
                          'no3':'industryNo3'})

df3.head(3)

,industry,industryNo,industry2,industryNo2,industry3,industryNo3
0,批發╱零售╱傳直銷業,1003000000,批發業,1003001000,農╱畜╱水產品批發業,1003001001
0,批發╱零售╱傳直銷業,1003000000,批發業,1003001000,食品什貨批發業,1003001002
0,批發╱零售╱傳直銷業,1003000000,批發業,1003001000,鞋類╱布類╱服飾品批發業,1003001003


In [7]:
df = pd.merge(df.rename(columns={'industry':'industry3', 'industryNo':'industryNo3'}),
              df3,
              how='left',
              on=['industry3', 'industryNo3'])
df

,jobName,appearDate,custName,custUrl,analysisUrl,hrName,email,visit,phone,acceptRole,...,industryArea,longitude,latitude,industry3,industryNo3,employees,industry,industryNo,industry2,industryNo2
0,"生產線作業員(起薪$30,000，含餐食) (加班、獎金另計)",2021/08/16,總信食品有限公司,https://www.104.com.tw/company/13vdp2c8,//www.104.com.tw/jobs/apply/analysis/10062,胡慧敏小姐,,,,"[{'code': 1, 'description': '上班族'}, {'code': 2...",...,台中工業區,120.606144,24.160068,食品什貨批發業,1003001002,95人,批發╱零售╱傳直銷業,1.003000e+09,批發業,1003001000
1,汐止分院 護理科 護理人員(內外科病房.產兒科病房.加護病房.門診.安寧病房.護理之家.急診),2021/08/17,國泰醫療財團法人國泰綜合醫院,https://www.104.com.tw/company/1waq7ko,//www.104.com.tw/jobs/apply/analysis/1006w,李小姐,sue@cgh.org.tw,,,"[{'code': 1, 'description': '上班族'}, {'code': 6...",...,,121.66141,25.072999,醫院,1012001001,3400人,醫療保健及環境衛生業,1.012000e+09,醫療服務業,1012001000
2,印刷見習技師,2021/08/16,康揚特殊美術印刷股份有限公司,https://www.104.com.tw/company/5ql7jgg,//www.104.com.tw/jobs/apply/analysis/1008q,邱先生,ford521213@yahoo.com.tw,,,"[{'code': 1, 'description': '上班族'}, {'code': 2...",...,,121.5910051,25.0553784,印刷業,1002006002,35人,一般製造業,1.002000e+09,印刷相關業,1002006000
3,證券營業員(南科分公司),2021/08/12,國票綜合證券股份有限公司,https://www.104.com.tw/company/at8koao,//www.104.com.tw/jobs/apply/analysis/100il,郭經理,,,,"[{'code': 1, 'description': '上班族'}, {'code': 2...",...,,120.3,23.0833333,證券及期貨業,1004002002,1056人,金融投顧及保險業,1.004000e+09,投資理財相關業,1004002000
4,汐止分院 護理科 內外科 專科護理師,2021/08/17,國泰醫療財團法人國泰綜合醫院,https://www.104.com.tw/company/1waq7ko,//www.104.com.tw/jobs/apply/analysis/100ry,李小姐,sue@cgh.org.tw,,,"[{'code': 1, 'description': '上班族'}, {'code': 6...",...,,121.66141,25.072999,醫院,1012001001,3400人,醫療保健及環境衛生業,1.012000e+09,醫療服務業,1012001000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271457,初級查帳審計人員,2021/08/17,聯緯聯合會計師事務所,https://www.104.com.tw/company/19uizocg,//www.104.com.tw/jobs/apply/analysis/zygi,陳小姐,,,03-6673668,"[{'code': 1, 'description': '上班族'}, {'code': 2...",...,,121.0278526,24.8230679,會計服務業,1008002001,38人,法律╱會計╱顧問╱研發╱設計業,1.008000e+09,會計服務業,1008002000
271458,業務儲備幹部 (保障前9個月每月5萬元) I01,2021/08/18,永慶房產集團_永慶房屋仲介股份有限公司,https://www.104.com.tw/company/agf7nww,//www.104.com.tw/jobs/apply/analysis/zz37,袁小姐,,,,"[{'code': 1, 'description': '上班族'}, {'code': 2...",...,,121.4970294,25.0262857,不動產經營業,1011004001,3000人,建築營造及不動產相關業,1.011000e+09,不動產業,1011004000
271459,塑膠設備操作員_磨粉機,2021/08/16,良輝塑膠工業股份有限公司,https://www.104.com.tw/company/vd2i754,//www.104.com.tw/jobs/apply/analysis/zzjx,鄭先生/潘小姐,,,06-2633110-31,"[{'code': 1, 'description': '上班族'}, {'code': 2...",...,安平工業區,120.182326,22.972978,塑膠製品製造業,1002009002,50人,一般製造業,1.002000e+09,橡膠及塑膠製品製造業,1002009000
271460,Java程式開發工程師,2021/08/17,藍科數位科技股份有限公司,https://www.104.com.tw/company/wjm2d5c,//www.104.com.tw/jobs/apply/analysis/zzsf,陳小姐,,,,"[{'code': 1, 'description': '上班族'}, {'code': 6...",...,,121.5801053,25.0643078,電腦軟體服務業,1001001002,90人,電子資訊╱軟體╱半導體相關業,1.001000e+09,軟體及網路相關業,1001001000


In [8]:
df.to_pickle('./data/JobDetail_20210818.pkl')

# Drop 230k jobs file

In [9]:
import os
import sys
import shutil
shutil.rmtree('/Users/TLYu0419/Documents/Github/DataScience/02_Data_Collenction/WebCrawler/HumanResource_104/data/JobDetail/')

# Analysis